In [1]:
import os
os.chdir("../../../")
from src.backtesting.run_backtest import prepare_and_run_backtest

result=prepare_and_run_backtest()
result

[*********************100%%**********************]  1 of 1 completed


{'sharpe_ratio': -58.66063557429906,
 'return': 0.0016255774658640904,
 'max_drawdown': 0.023526728407521492,
 'win_trade': 7,
 'loss_trade': 3,
 'total_trade': 11,
 'start_portfolio': 100000,
 'final_portfolio': 100162.6899433136}

In [3]:
from sqlalchemy.orm import sessionmaker
from models import engine, BacktestResult, Dim_Date
from datetime import datetime

# The backtest results dictionary
results = {
    'sharpe_ratio': -58.66063557429906,
    'return': 0.0016255774658640904,
    'max_drawdown': 0.023526728407521492,
    'win_trade': 7,
    'loss_trade': 3,
    'total_trade': 11,
    'start_portfolio': 100000,
    'final_portfolio': 100162.6899433136
}

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Correctly querying Dim_Date and handling the result
current_date = datetime.now().date()
date_record = session.query(Dim_Date).filter_by(Date=current_date).first()
if not date_record:
    date_record = Dim_Date(Date=current_date)
    session.add(date_record)
    session.commit()

# Create a new BacktestResult record
backtest_result = BacktestResult(
    SharpeRatio=results['sharpe_ratio'],
    TotalReturn=results['return'],
    MaxDrawdown=results['max_drawdown'],
    WinningTrades=results['win_trade'],
    LosingTrades=results['loss_trade'],
    TradeCount=results['total_trade'],
    DateKey=date_record.DateKey,
    StartPortfolio=results['start_portfolio'],
    FinalPortfolio=results['final_portfolio']
)

# Add the new record to the session and commit it
session.add(backtest_result)
session.commit()

print("SUCCESS - Backtest result added to the database.")

SUCCESS - Backtest result added to the database.
